In [21]:
# data processing
import pandas as pd
import geopandas as gpd
import numpy as np
import carto2gpd
import esri2gpd

# basic graphics
from matplotlib import pyplot as plt
import holoviews as hv
import hvplot.pandas
import altair as alt
import folium
import matplotlib.colors as mcolors

# advanced graphics
import panel as pn
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import param
import folium as fm

# census
import cenpy

# osm
import osmnx as ox
# bus ridership data
from sodapy import Socrata

# analysis
import math as mt


# shapely
from shapely.geometry import Polygon, LineString, Point
from sklearn.model_selection import GridSearchCV

import pandas as pd

In [48]:
philly_data = gpd.read_file('data/philly_data_with_predictions.geojson').set_crs("epsg:3857", allow_override=True,)
philly_stops = pd.read_csv("data/stops.txt")

In [51]:
philly_data=philly_data.to_crs("epsg:4326")

In [52]:
def init_map(lat=39.99, long=-75.13, zoom_start=11):
    return fm.Map(location=[lat,long], zoom_start=zoom_start)

In [53]:
data_test = philly_data.to_crs(epsg=4326)
bus_ridership = philly_stops

In [54]:
## choropleth for predictions
def choose_x(x='predictions'):    
    # Minimum
    min_val = data_test[x].min()

    # Maximum
    max_val = data_test[x].max()

    # Calculate a normalized column
    normalized = (data_test[x] - min_val) / (max_val - min_val)

    # Add to the dataframe
    data_test["normalized"] = normalized
 
# Use a red-purple colorbrewer color scheme
cmap = plt.get_cmap('PuBu')

def get_style(feature):
    # Get the data value from the feature
    value = feature['properties']['normalized']

    # Evaluate the color map
    # NOTE: value must between 0 and 1
    rgb_color = cmap(value) # this is an RGB tuple

    # Convert to hex string
    color = mcolors.rgb2hex(rgb_color)

    # Return the style dictionary
    return {'weight': 0.5, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

def get_highlighted_style(feature):
    return {"weight": 3, "color": "black"}
    

In [55]:
columns = list(data_test.columns[3:])
columns.remove('GEO_ID')
columns.remove('state')
columns.remove('county')
columns.remove('tract')
columns.remove('block group')


class PanelFoliumMap(param.Parameterized):
    feature = param.Selector(default="predictions", objects=columns)

    def __init__(self, max_records=2500, name=""):
        super().__init__(name=name)

        TOR_COORDINATES = (
            pd.to_numeric(bus_ridership["stop_lat"]).mean(),
            pd.to_numeric(bus_ridership["stop_lon"]).mean(),
        )

        # subset to match subset of locations
        self.mc = MarkerCluster()

        # iterate through dataset to create clusters
        for row in bus_ridership[0:max_records].itertuples():
            self.mc.add_child(
                fm.Marker(location=[row.stop_lat, row.stop_lon], 
                          #popup=row.predictions
                         )
            )
            
    # Set default choropleth column
    choose_x()

    @param.depends("feature")
    def get_map(self):

        # Get new map
        self.map = init_map()
        
        # Add markercluster
        self.map.add_child(self.mc)
        
        # Set choropleth column
        choose_x(self.feature)


        # Add the GeoJson to the map
        folium.GeoJson(
            data_test[
                ["geometry", "normalized", self.feature]
            ].to_json(),  # IMPORTANT: make sure CRS is lat/lng (EPSG=4326)
            style_function=get_style,
            tooltip=folium.GeoJsonTooltip([self.feature]),
        ).add_to(self.map)

        return pn.Pane(self.map, width=1000, height=500)

In [56]:
app = PanelFoliumMap(name="")

In [57]:
# The title
title = pn.Pane("<h1>Predicted Bus Ridership in Philadelphia by Block Group</h1>", width=1000)
# Layout the dashboard
panel = pn.Column(
    pn.Row(title),
    pn.Row(pn.Column(app.param.feature, app.get_map)),
)
panel.servable()

Column
    [0] Row
        [0] Markdown(str, width=1000)
    [1] Row
        [0] Column
            [0] Select(name='Feature', options=OrderedDict([('median_rent...]), value='predictions')
            [1] ParamMethod(method)